In [42]:
from datetime import datetime, time, timedelta

import pandas as pd

from ib_async.ib import IB
from ib_async.contract import Stock
from ib_async import util

from tqdm.auto import tqdm

### Connect IB


In [ ]:
util.startLoop()
ib = IB()
ib.connect("127.0.0.1", 4001, clientId=1)

<IB connected to 127.0.0.1:4001 clientId=1>

In [ ]:
news_df = pd.read_csv("/Users/akseljoonas/Documents/predtrade/news_data_nov.csv")

### Validate tickers


In [ ]:
import yaml
def load_config(sector = 'biotech'):
    config_file = f"{sector}.yaml"
    with open(config_file, "r") as file:
        rss_dict = yaml.safe_load(file)

    return rss_dict

In [ ]:
rss_dict = load_config()
valid_tickers = []
for ticker in rss_dict.keys():
    contract = Stock(ticker, "SMART", "USD")
    if ib.qualifyContracts(contract):
        valid_tickers.append(ticker)
        
for ticker in rss_dict.keys():
    if ticker not in valid_tickers:
        print(ticker)
print(len(rss_dict.keys()), len(valid_tickers))

remove invalid tickers from df


In [ ]:
news_df = news_df[news_df["ticker"].isin(valid_tickers)]


In [ ]:
news_df.to_csv("/Users/akseljoonas/Documents/predtrade/news_data_nov_validated.csv", index=False)

### Turn into eastern time

In [ ]:
import pytz

utc = pytz.utc
eastern = pytz.timezone("US/Eastern")

# Convert 'published_gmt' from UTC to Eastern Time
news_df["published_gmt"] = news_df["published_gmt"].apply(
    lambda x: datetime.fromisoformat(x)
    .astimezone(eastern)
    .strftime("%Y-%m-%d %H:%M:%S%z")
)
news_df.head()

In [ ]:
news_df.to_csv(
    "/Users/akseljoonas/Documents/predtrade/news_data_nov_validated.csv", index=False
)

### Import prices

In [44]:
def get_day_price(ticker, start_time):
    contract = Stock(ticker, "SMART", "USD")
    if not ib.qualifyContracts(contract):
        return pd.DataFrame()

    date = datetime.strptime(start_time, "%Y-%m-%d %H:%M:%S%z").date()
    end_of_day = datetime.combine(date, time(23, 59, 59))

    bar_data = ib.reqHistoricalData(
        contract,
        endDateTime=end_of_day.strftime("%Y%m%d %H:%M:%S"),
        durationStr="2 D",
        barSizeSetting="1 min",
        whatToShow="TRADES",
        useRTH=False,
    )
    bar_data = util.df(bar_data)

    if bar_data is None or bar_data.empty:

        end_of_day = end_of_day + timedelta(days=1)
        bar_data = ib.reqHistoricalData(
            contract,
            endDateTime=end_of_day.strftime("%Y%m%d %H:%M:%S"),
            durationStr="3 D",
            barSizeSetting="1 min",
            whatToShow="TRADES",
            useRTH=False,
        )
        bar_data = util.df(bar_data)
        if bar_data is None or bar_data.empty:
            bar_data = pd.DataFrame()

    return bar_data

In [45]:
import yfinance as yf


def get_yf_data(ticker, start_date):
    try:
        stock = yf.Ticker(ticker)
    except Exception:
        return None, None
    info = stock.info
    if info is None:
        return None, None

    try:
        float_shares = info["sharesOutstanding"]
    except KeyError:
        float_shares = None

    try:
        short_interest = info["dateShortInterest"]
    except KeyError:
        short_interest = None

    return float_shares, short_interest

In [46]:
def get_avg_volume(ticker, start_time):
    contract = Stock(ticker, "SMART", "USD")

    bar_data = ib.reqHistoricalData(
        contract,
        endDateTime=start_time.strftime("%Y%m%d %H:%M:%S"),
        durationStr="10 D",
        barSizeSetting="1 day",
        whatToShow="TRADES",
        useRTH=False,
    )
    
    if not bar_data:
        return 0
    
    bar_data = util.df(bar_data)
    return int(bar_data["volume"].mean())

In [47]:
def get_ibb_trend(start_time):
    # Calculate the start of the week

    contract = Stock("IBB", "SMART", "USD")
    bar_data = ib.reqHistoricalData(
        contract,
        endDateTime=start_time.strftime("%Y%m%d %H:%M:%S"),
        durationStr="10 D",
        barSizeSetting="1 day",
        whatToShow="TRADES",
        useRTH=False,
    )
    bar_data = util.df(bar_data)

    if bar_data.empty:
        return None  # No data available

    # Calculate the trend
    first_price = bar_data.iloc[0]["close"]
    last_price = bar_data.iloc[-1]["close"]

    # Return 1 if trend is positive, 0 otherwise
    return 1 if last_price > first_price else 0

In [48]:
def get_times(published_time):
    tzinfo = published_time.tzinfo
    # Define market times in EST
    premarket_start = time(0, 0, tzinfo=tzinfo)
    premarket_end = time(9, 30, tzinfo=tzinfo)
    market_start = time(9, 30, tzinfo=tzinfo)
    market_end = time(16, 0, tzinfo=tzinfo)
    after_hours_start = time(16, 0, tzinfo=tzinfo)
    after_hours_end = time(23, 59, 59, tzinfo=tzinfo)

    # Get the date component of the published_time
    published_date = published_time.date()
    prev_date = published_time - timedelta(days=1)
    next_date = published_time + timedelta(days=1)

    time_tz = published_time.time().replace(tzinfo=published_time.tzinfo)

    if premarket_start < time_tz < premarket_end:
        start_time = pd.to_datetime(datetime.combine(prev_date, market_end))
        end_time = pd.to_datetime(datetime.combine(published_time, market_start))
    elif market_start <= time_tz < market_end:
        start_time = published_time
        end_time = published_time
    elif after_hours_start <= time_tz < after_hours_end:
        start_time = pd.to_datetime(datetime.combine(published_time, market_end))
        end_time = pd.to_datetime(datetime.combine(next_date, market_start))
    else:
        raise ValueError("Invalid time period for trading hours")

    
    return start_time, end_time

In [49]:
def format_bars(df, target_time, ticker):
    target_time = pd.to_datetime(target_time)

    start_time, end_time = get_times(target_time)

    start_time = start_time - timedelta(minutes=1)
    end_time = end_time + timedelta(minutes=5)

    print(target_time)
    print(start_time)
    print(end_time)

    day_df = df
    daily_volume = day_df["volume"].sum()

    # Get the time needed
    df = df[(df["date"] >= start_time) & (df["date"] <= end_time)]
    if df.empty:
        return {
            "open": None,
            "high": None,
            "low": None,
            "close": None,
            "average": None,
            "volume": None,
            "trade_count": None,
            "vwap": None,
            "relative_volume_daily": None,
            "relative_volume_5m": None,
            "float_rotation": None,
            "short_interest": None,
            "index_trend": None,
            "price_move": None,
            "increased_3_percent": None,
        }


    float_shares, short_interest = get_yf_data(ticker, start_time)
    avg_daily_volume = get_avg_volume(ticker, start_time)
    index_trend = get_ibb_trend(start_time)

    first_close = df["close"].iloc[0]
    first_open = df["open"].iloc[0]
    last_close = df["close"].iloc[-1]
    max_high = df["high"].max()
    min_low = df["low"].min()
    average = df["average"].mean()
    sum_volume = df["volume"].sum()
    sum_trade_count = df["barCount"].sum()

    relative_volume_daily = (
        daily_volume - avg_daily_volume
    ) / avg_daily_volume + 0.00000000000000001
    prev_5m_volume = day_df[
        (day_df["date"] >= target_time - pd.Timedelta(minutes=6))
        & (day_df["date"] <= start_time)
    ]["volume"].mean()
    relative_volume_5m = (
        sum_volume - prev_5m_volume
    ) / prev_5m_volume + 0.00000000000000001
    float_rotation = daily_volume / float_shares

    price_move = (last_close - first_close) / first_close

    print(f"price move {price_move}")

    increased_3_percent = 1 if price_move >= 0.03 else 0

    pv = df["average"] * df["volume"]
    cumulative_pv = sum(pv)
    aggregated_vwap = cumulative_pv / sum_volume

    return {
        "open": first_open,
        "high": max_high,
        "low": min_low,
        "close": last_close,
        "average": average,
        "volume": sum_volume,
        "trade_count": sum_trade_count,
        "vwap": aggregated_vwap,
        "relative_volume_daily": relative_volume_daily,
        "relative_volume_5m": relative_volume_5m,
        "float_rotation": float_rotation,
        "short_interest": short_interest,
        "index_trend": index_trend,
        "price_move": price_move,
        "increased_3_percent": increased_3_percent,
    }

In [50]:
import math


def add_stock_data(df):
    print(len(df))
    for index, row in tqdm(df.iterrows()):
        if index < 3500 or not math.isnan(row["increased_3_percent"]):
            continue
        ticker = row["ticker"]

        start_time = row["published_gmt"]
        day_df = get_day_price(ticker, start_time)

        if not day_df.empty:
            print(f"\n\n{index} {ticker}")
            stock_info = format_bars(day_df, start_time, ticker)

            # Update row
            for col in stock_info.keys():
                df.loc[index, col] = stock_info[col]

        if index % 100 == 0:
            df.to_csv("/Users/akseljoonas/Documents/predtrade/stock_bars.csv", index=False)

    return df

In [51]:
import os


if os.path.exists("/Users/akseljoonas/Documents/predtrade/stock_bars.csv"):
    news_df = pd.read_csv("/Users/akseljoonas/Documents/predtrade/stock_bars.csv")


stock_df = add_stock_data(news_df)

stock_df.dropna(subset=["increased_3_percent"], axis=0, inplace=True)
stock_df["topic"] = (
    stock_df["topic"]
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("/", "_")
    .str.replace("&", "")
    .str.replace("'", "")
)

4122


0it [00:00, ?it/s]



3521 MYCOF
2023-10-10 02:00:00-04:00
2023-10-09 15:59:00-04:00
2023-10-10 09:35:00-04:00


3526 MYCOF
2023-05-29 19:42:00-04:00
2023-05-29 15:59:00-04:00
2023-05-30 09:35:00-04:00


3558 TOVX
2024-08-17 12:58:00-04:00
2024-08-17 12:57:00-04:00
2024-08-17 13:03:00-04:00


3569 PCVX
2024-09-02 18:40:00-04:00
2024-09-02 15:59:00-04:00
2024-09-03 09:35:00-04:00


3601 BNTX
2024-08-22 14:15:00-04:00
2024-08-22 14:14:00-04:00
2024-08-22 14:20:00-04:00
price move -0.003171367085740187


3602 BNTX
2024-08-16 06:45:00-04:00
2024-08-15 15:59:00-04:00
2024-08-16 09:35:00-04:00
price move -0.023708624441644798


3603 BNTX
2024-08-16 06:45:00-04:00
2024-08-15 15:59:00-04:00
2024-08-16 09:35:00-04:00
price move -0.023708624441644798


3604 CADL
2024-09-03 08:00:00-04:00
2024-09-02 15:59:00-04:00
2024-09-03 09:35:00-04:00
price move 0.0028653295128939216


3605 CADL
2024-08-13 08:00:00-04:00
2024-08-12 15:59:00-04:00
2024-08-13 09:35:00-04:00
price move -0.0017889087656529135


3606 RAPT
2024-08-08

Error 162, reqId 6: Historical Market Data Service error message:HMDS query returned no data: MYCOF@SMART Trades, contract: Stock(conId=557337337, symbol='MYCOF', exchange='SMART', primaryExchange='PINK', currency='USD', localSymbol='MYCOF', tradingClass='CURRENT')
Error 162, reqId 7: Historical Market Data Service error message:HMDS query returned no data: MYCOF@SMART Trades, contract: Stock(conId=557337337, symbol='MYCOF', exchange='SMART', primaryExchange='PINK', currency='USD', localSymbol='MYCOF', tradingClass='CURRENT')
Error 162, reqId 417: Historical Market Data Service error message:No market data permissions for ARCAEDGE STK, contract: Stock(conId=350194771, symbol='BICX', exchange='SMART', primaryExchange='PINK', currency='USD', localSymbol='BICX', tradingClass='BICX')
Error 162, reqId 418: Historical Market Data Service error message:No market data permissions for ARCAEDGE STK, contract: Stock(conId=350194771, symbol='BICX', exchange='SMART', primaryExchange='PINK', currenc

/var/folders/j4/cvc05xl96vd2tzttrm50lzb00000gn/T/ipykernel_7315/1564073674.py:58: RuntimeWarning: divide by zero encountered in scalar divide
  relative_volume_5m = (


...

price move 0.007692307692307699


4047 NXTC
2024-10-04 09:05:00-04:00
2024-10-03 15:59:00-04:00
2024-10-04 09:35:00-04:00
price move -0.007462686567164185


4048 ETNB
2024-11-07 16:05:00-05:00
2024-11-07 15:59:00-05:00
2024-11-08 09:35:00-05:00
price move -0.01706827309236965


4049 ETNB
2024-11-05 16:05:00-05:00
2024-11-05 15:59:00-05:00
2024-11-06 09:35:00-05:00
price move 0.0


4050 ETNB
2024-10-15 08:30:00-04:00
2024-10-14 15:59:00-04:00
2024-10-15 09:35:00-04:00
price move 0.0012642225031605293


4051 ETNB
2024-10-11 16:05:00-04:00
2024-10-11 15:59:00-04:00
2024-10-12 09:35:00-04:00
price move 0.001272477217516487


4052 NKTX
2024-11-07 16:02:00-05:00
2024-11-07 15:59:00-05:00
2024-11-08 09:35:00-05:00
price move 0.0


4053 GLUE
2024-11-07 07:00:00-05:00
2024-11-06 15:59:00-05:00
2024-11-07 09:35:00-05:00
price move 0.007399577167018869


4054 GLUE
2024-10-28 07:00:00-04:00
2024-10-27 15:59:00-04:00
2024-10-28 09:35:00-04:00
price move -0.13279132791327908


4055 GLUE
2024-10-23 0

/var/folders/j4/cvc05xl96vd2tzttrm50lzb00000gn/T/ipykernel_7315/1564073674.py:58: RuntimeWarning: divide by zero encountered in scalar divide
  relative_volume_5m = (


price move 0.16394160583941597


4106 APTO
2024-11-08 16:01:00-05:00
2024-11-08 15:59:00-05:00
2024-11-09 09:35:00-05:00


/var/folders/j4/cvc05xl96vd2tzttrm50lzb00000gn/T/ipykernel_7315/1564073674.py:58: RuntimeWarning: divide by zero encountered in scalar divide
  relative_volume_5m = (


price move -0.012467532467532535


4107 FGEN
2024-11-04 16:05:00-05:00
2024-11-04 15:59:00-05:00
2024-11-05 09:35:00-05:00
price move 0.017685386286069


4108 GLPG
2024-11-05 16:01:00-05:00
2024-11-05 15:59:00-05:00
2024-11-06 09:35:00-05:00
price move -0.005609573672400845


4109 GLPG
2024-11-05 16:01:00-05:00
2024-11-05 15:59:00-05:00
2024-11-06 09:35:00-05:00
price move -0.005609573672400845


4110 GLPG
2024-10-30 16:01:00-04:00
2024-10-30 15:59:00-04:00
2024-10-31 09:35:00-04:00
price move 0.0


4111 GLPG
2024-10-30 16:01:00-04:00
2024-10-30 15:59:00-04:00
2024-10-31 09:35:00-04:00
price move 0.0


4112 GLPG
2024-10-07 01:00:00-04:00
2024-10-06 15:59:00-04:00
2024-10-07 09:35:00-04:00
price move 0.0033277870216305446


4113 GLPG
2024-10-07 01:00:00-04:00
2024-10-06 15:59:00-04:00
2024-10-07 09:35:00-04:00
price move 0.0033277870216305446


4114 TCRX
2024-11-05 09:15:00-05:00
2024-11-04 15:59:00-05:00
2024-11-05 09:35:00-05:00
price move -0.0014566929133857536


4115 TCRX
2024-10-04

In [52]:
stock_df.head()

,Unnamed: 0.1,Unnamed: 0,published_gmt,ticker,title,summary,description,link,language,topic,...,volume,trade_count,vwap,relative_volume_daily,relative_volume_5m,float_rotation,short_interest,index_trend,price_move,increased_3_percent
0,0.0,0.0,2020-03-30 10:17:00-0400,GRFS,GigaGen Initiates Development of Recombinant P...,"SOUTH SAN FRANCISCO, Calif., March 30, 2020 ...","SOUTH SAN FRANCISCO, Calif., March 30, 2020 ...",https://www.globenewswire.com/news-release/202...,NaN,company_announcement,...,12226.0,100.0,20.195202,-0.120647,3.298623,0.004089,1.728950e+09,1.0,0.000000,0.0
1,1.0,1.0,2020-07-29 09:30:00-0400,GRFS,GigaGen to Present at the Inaugural LifeSci Pa...,"SOUTH SAN FRANCISCO, Calif., July 29, 2020 (...","SOUTH SAN FRANCISCO, Calif., July 29, 2020 (...",https://www.globenewswire.com/news-release/202...,NaN,company_announcement,...,13469.0,49.0,19.190655,0.676953,inf,0.005237,1.728950e+09,0.0,-0.005720,0.0
2,2.0,2.0,2020-08-10 10:17:00-0400,GRFS,GigaGen Announces Publication of Research Desc...,"SOUTH SAN FRANCISCO, Calif., Aug. 10, 2020 (...","SOUTH SAN FRANCISCO, Calif., Aug. 10, 2020 (...",https://www.globenewswire.com/news-release/202...,NaN,company_announcement,...,11919.0,62.0,17.949867,0.898022,11.788627,0.004761,1.728950e+09,0.0,-0.001669,0.0
3,3.0,3.0,2020-09-09 10:17:00-0400,GRFS,GigaGen Initiates Large-Scale Manufacturing of...,"SOUTH SAN FRANCISCO, Calif., Sept. 09, 2020 ...","SOUTH SAN FRANCISCO, Calif., Sept. 09, 2020 ...",https://www.globenewswire.com/news-release/202...,NaN,product___services_announcement,...,7757.0,69.0,16.591955,1.120598,0.409851,0.008041,1.728950e+09,0.0,0.001808,0.0
4,4.0,4.0,2020-10-20 10:17:00-0400,GRFS,GigaGen Announces Publication in the Peer-Revi...,Article Details GigaGen’s Affinity Maturation ...,Article Details GigaGen’s Affinity Maturation ...,https://www.globenewswire.com/news-release/202...,NaN,health,...,66644.0,393.0,19.644508,1.182335,29.622147,0.008439,1.728950e+09,1.0,0.001535,0.0


In [53]:
# def get_ta_indicators(df):
#     EMA = abstract.Function("ema")
#     BBANDS = abstract.Function("bbands")
#     MACD = abstract.Function("macd")
#     RSI = abstract.Function("rsi")

#     macd, macdsignal, macdhist = MACD(
#         df["close"], fastperiod=12, slowperiod=26, signalperiod=9
#     )
#     bb_upper, bb_middle, bb_lower = BBANDS(
#         df["close"], timeperiod=5, nbdevup=2, nbdevdn=2, matype=0
#     )
#     real = RSI(df["close"], timeperiod=14)

#     ema_21 = EMA(df["close"], timeperiod=21)
#     ema_100 = EMA(df["close"], timeperiod=100)
#     ema_7 = EMA(df["close"], timeperiod=7)

#     exit(1)

In [54]:
stock_df.to_csv("/Users/akseljoonas/Documents/predtrade/stock_bars_final.csv", index=False)